In [21]:
PATHS = (
    ("/tmp/numered_all.json", "/tmp/gpt4o-generated-speech"),
    ("/tmp/ggpt_numbered.json", "/tmp/groundinggpt-generated-speech"),
    ("/tmp/ggpt_numbered2.json", "/tmp/groundinggpt-generated-speech2")
)

In [1]:
import json

with open("/tmp/numered_all.json") as inf:
    data = json.load(inf)

In [9]:
from string import punctuation

punct = set(punctuation)

In [38]:
def gather_phrase(text):
    words = text.split(" ")
    phrase = []
    inphrase = False
    for word in words:
        if inphrase:
            if word.endswith("*") or (word[-1] in punct and word[-2] == "*"):
                word = word.replace("*", "")
                phrase.append(word)
                inphrase = False
            else:
                phrase.append(word)
        else:
            if word.startswith("*"):
                if word.endswith("*"):
                    phrase.append(word.strip("*"))
                    inphrase = False
                else:
                    phrase.append(word.strip("*"))
                inphrase = True
            else:
                continue
    return (" ".join(phrase)).strip("*")

In [ ]:
def index_of_determiner(phrase):
    dp = gather_phrase(phrase)
    words = phrase.split(" ")
    for i, word in enumerate(words):
        if word in ["this", "that", "these", "those"]:
            return i
    return -1

In [ ]:
def clean_text(text):
    text = text.replace("’", "'")
    text = text.replace("—", " ")
    words = text.split(" ")
    cleaned = [w.lower().strip(punctuation) for w in words]
    return " ".join(cleaned)

In [23]:
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.6 MB/s eta 0:00:00:00:010:01m
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7


In [24]:
from jiwer import wer

In [22]:
def prune_fillers(text):
    FILLERS = ["uh", "um"]
    words = [x for x in text.split(" ") if x not in FILLERS]
    return " ".join(words)

In [32]:
FIXES = {
    "hsi_4_0717_222_002__0__8": ("racket", "raquet")
}

In [37]:
from pathlib import Path

THRESHOLD = 0.3

for item in data:
    person = item["person"]
    fileid = item["id"]
    text = item["snippet"]

    tsvfile = Path("/tmp") / f"gpt4o-generated-speech" / f"{fileid}.tsv"

    tsvdata = []
    with open(tsvfile) as inf:
        for line in inf:
            line = line.strip()
            if not line:
                continue
            tsvdata.append(line.split("\t"))
    
    tsvwords = [x[2] for x in tsvdata]
    tsvtext = " ".join(tsvwords)

    cleaned_text = clean_text(text.strip())
    cleaned_tsv = clean_text(tsvtext)

    phrase = gather_phrase(text)

    if cleaned_text != cleaned_tsv:
        if cleaned_tsv.replace("-", " ") == cleaned_text:
            cleaned_tsv = cleaned_tsv.replace("-", " ")

    discard_ids = []

    if cleaned_text != cleaned_tsv:
        if not phrase in cleaned_tsv:
            print("Phrase not in tsv", phrase)
            discard_ids.append(fileid)
        print("Mismatch", fileid, person)
        print(cleaned_text)
        print(cleaned_tsv)
        cur_wer = wer(cleaned_text, cleaned_tsv)
        print(cur_wer)
        if cur_wer > THRESHOLD:
            discard_ids.append(fileid)
            print("WER above threshold, discarding")
        print()

    # print(gather_phrase(text), "\t", text)


Phrase not in tsv it* up near the window, you might find working on projects more enjoyable.
Mismatch hsi_4_0717_222_002__0__9 4
if we set it up near the window you might find working on projects more enjoyable
wall if we set up near the window you might find working on projects more enjoyable
0.125

Phrase not in tsv it* comes with a weekend edition, you should take advantage of the special offers.
Mismatch hsi_4_0717_222_002__0__14 4
if it comes with a weekend edition you should take advantage of the special offers
it comes with a weekend edition you should take advantage of the special offers
0.06666666666666667

Mismatch hsi_4_0717_222_002__0__21 4
admire how this work complements the colors in the room adding so much character
i admire how this work complements the colors in the room adding so much character
0.07142857142857142

Mismatch hsi_4_0717_222_002__0__26 4
let's try moving that couch to a different spot and see if it opens up more space it might make the room feel cozier
